In [1]:
import numpy as np
import pandas as pd
import torch

In [5]:
def preprocess():
    data = pd.read_csv("/mnt/d/mlpp/nyse/data_fund.csv")
    fun = data.loc[~data['Indicator Name'].isin(['Common Shares Outstanding','Share Price'])].reset_index(drop=True)
    fun['yyyymm']=pd.to_datetime(fun['publish date'])
    fun = pd.pivot_table(fun, values='Indicator Value', index=['Ticker', 'yyyymm'],columns=['Indicator Name'], aggfunc=np.mean).reset_index()

    to_remove = ['Ticker','yyyymm','Avg. Basic Shares Outstanding','Avg. Diluted Shares Outstanding','Total Assets']
    fun_cols = list(fun.columns)
    for elem in to_remove:
        fun_cols.remove(elem)

    fun[fun_cols] = fun[fun_cols].div(fun['Total Assets'], axis=0).reset_index(drop=True)

    features = fun[fun_cols+['yyyymm','Ticker']]
    features = features.dropna().reset_index(drop=True)
    features = features.fillna(0).reset_index(drop=True)

    zscore = lambda x: (x - x.mean()) / x.std()
    final = features[['Ticker','yyyymm']]
    for item in fun_cols:
        temp = features.groupby([features.yyyymm])[item].transform(zscore)
        final = pd.concat([final,temp], axis=1).reset_index(drop=True)

    data['publish date']=pd.to_datetime(data['publish date'])
    rtns = data.loc[data['Indicator Name']=='Share Price', ['Ticker','publish date','Company Industry Classification Code', 'Indicator Value']].reset_index(drop=True)
    rtns.columns = ['Ticker','Date','Industry','Price']
    rtns['Rtn1d'] = rtns.groupby('Ticker')['Price'].apply(lambda x: np.log(x).diff())
    rtns['Rtn1q'] = rtns.groupby('Ticker')['Rtn1d'].rolling(63).sum().shift(-63).reset_index(0,drop=True)
    rtns['Rtn1d'] = rtns.groupby('Ticker')['Rtn1d'].shift(-1).reset_index(0,drop=True)
    rtns.dropna(inplace=True)

    final = final.merge(rtns, left_on=['Ticker','yyyymm'], right_on =['Ticker','Date'],how='inner').reset_index(0,drop=True)
    final.dropna(inplace=True)
    train = final[final['yyyymm']<='2016-12-31'].reset_index(0,drop=True)
    test = final[final['yyyymm']>'2016-12-31'].reset_index(0,drop=True)

    X_train = train[fun_cols].values
    y_train = train['Rtn1q'].values
    X_test = test[fun_cols].values
    y_test = test['Rtn1q'].values
    X_tr = torch.tensor(X_train, dtype=torch.float)
    y_tr = torch.tensor(y_train, dtype=torch.float)
    X_ts = torch.tensor(X_test, dtype=torch.float)
    y_ts = torch.tensor(y_test, dtype=torch.float)

    return train

In [6]:
data = preprocess()

/mnt/c/Users/aksmi/Desktop/Drive/Fall '19/Courses/ML with Probabilistic Programming/course_repo/probprog/lib/python3.6/site-packages/pandas/core/series.py:853: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/mnt/c/Users/aksmi/Desktop/Drive/Fall '19/Courses/ML with Probabilistic Programming/course_repo/probprog/lib/python3.6/site-packages/pandas/core/algorithms.py:1977: RuntimeWarning: invalid value encountered in subtract
  out_arr[res_indexer] = arr[res_indexer] - arr[lag_indexer]


In [22]:
orig_cols = list(data.columns)
orig_cols

['Ticker',
 'yyyymm',
 'Accounts Payable',
 'COGS',
 'Cash From Financing Activities',
 'Cash From Investing Activities',
 'Cash From Operating Activities',
 'Cash and Cash Equivalents',
 'Current Assets',
 'Current Liabilities',
 'Dividends',
 'EBIT',
 'EBITDA',
 'Equity Before Minorities',
 'Goodwill',
 'Intangible Assets',
 'Long Term Debt',
 'Minorities',
 'Net Change in Cash',
 'Net Income from Discontinued Op.',
 'Net PP&E',
 'Net Profit',
 'Preferred Equity',
 'Receivables',
 'Retained Earnings',
 'Revenues',
 'SG&A',
 'Share Capital',
 'Short term debt',
 'Total Equity',
 'Total Liabilities',
 'Total Noncurrent Assets',
 'Total Noncurrent Liabilities',
 'Treasury Stock',
 'Date',
 'Industry',
 'Price',
 'Rtn1d',
 'Rtn1q']

In [17]:
cor = data.corr()
df1 = cor.stack().reset_index()
df1.columns = ['v1','v2','correlation']

In [21]:
df1[(np.abs(df1.correlation) > 0.7) & (df1.v1 != df1.v2)]

,v1,v2,correlation
59,COGS,Revenues,0.920643
153,Cash From Operating Activities,EBIT,0.790660
154,Cash From Operating Activities,EBITDA,0.822364
186,Cash and Cash Equivalents,Current Assets,0.701276
221,Current Assets,Cash and Cash Equivalents,0.701276
245,Current Assets,Total Noncurrent Assets,-0.994792
328,EBIT,Cash From Operating Activities,0.790660
334,EBIT,EBITDA,0.960648
343,EBIT,Net Profit,0.793430
364,EBITDA,Cash From Operating Activities,0.822364


In [27]:
to_remove = ['COGS', 'EBIT', 'Cash From Operating Activities', 'Cash and Cash Equivalents', 
             'Net Profit', 'Equity Before Minorities', 'Total Noncurrent Liabilities', 'Total Equity', 
             'Total Noncurrent Assets', 'Industry', 'Price', 'Retained Earnings']
new_data = data.drop(columns=to_remove)
new_data

,Ticker,yyyymm,Accounts Payable,Cash From Financing Activities,Cash From Investing Activities,Current Assets,Current Liabilities,Dividends,EBITDA,Goodwill,...,Receivables,Revenues,SG&A,Share Capital,Short term debt,Total Liabilities,Treasury Stock,Date,Rtn1d,Rtn1q
0,A,2011-09-07,-0.684551,-1.691360,2.183893,0.121045,-0.742225,1.040803,-0.303176,1.084159,...,0.277094,-0.565718,0.020479,2.773964,-0.558649,-0.096951,-1.484969,2011-09-07,-0.044433,3.571075e-02
1,A,2012-09-05,-0.709434,0.260642,-1.493651,-0.534877,-0.282426,0.376062,-0.066381,0.881559,...,0.006782,-0.920759,-0.866846,-0.227864,1.056813,-0.129860,-3.265011,2012-09-05,0.019993,4.462735e-02
2,A,2013-09-06,-1.033439,0.065106,0.645516,-0.383689,-0.593869,0.458310,-0.356593,0.940296,...,-0.116874,-0.907514,-0.024056,0.307529,-0.606792,0.160705,-1.309986,2013-09-06,0.011084,1.227928e-01
3,A,2014-03-05,-0.650589,-0.408724,-0.077143,-0.469958,-0.854185,-0.301951,0.158868,0.770338,...,-0.473313,-0.905179,-0.689553,-0.232574,-0.647795,-0.156960,-3.804480,2014-03-05,0.016550,-3.799878e-03
4,A,2015-09-02,-0.856860,-0.671772,0.926701,-0.967320,-0.941585,-0.463685,0.428915,1.616694,...,0.153932,-0.691456,-0.549809,0.096287,-0.528900,-0.453045,-1.788469,2015-09-02,0.006125,1.316645e-01
5,A,2016-03-08,-0.173059,-0.531663,0.284142,-0.373582,-0.343959,-0.256045,0.476688,1.401470,...,0.400335,-0.030090,-0.326312,-0.290124,-0.104606,-0.340633,-3.692599,2016-03-08,0.000814,1.859333e-01
6,A,2016-06-07,-1.221690,0.217149,-0.104873,0.156748,-0.720148,-0.064178,-0.072688,0.673351,...,-0.137396,-0.894683,-0.594274,0.859346,-0.149893,-0.612667,-2.713570,2016-06-07,0.008749,2.843372e-02
7,AAL,2013-02-20,-0.488844,0.115546,-0.074952,-0.559020,0.907997,0.867162,-0.397474,-0.737167,...,-0.528130,-0.175584,-0.830280,-0.308529,1.692750,2.033255,0.401327,2013-02-20,0.003147,2.976792e-01
8,AAL,2014-04-24,-0.367192,1.412635,-0.126090,-0.054538,0.599847,0.884878,-1.185748,-0.501138,...,-0.630098,-0.092270,-0.805956,-0.080330,-0.116702,0.665966,0.483756,2014-04-24,-0.034407,1.236533e-01
9,AAL,2014-07-24,-0.169053,-0.156052,0.161334,-0.163304,-0.152654,0.924150,0.144834,-0.252192,...,-0.346747,0.598950,-0.198142,-0.196075,-0.161195,-0.131850,0.506239,2014-07-24,-0.011523,-1.270412e-01


In [28]:
cor = new_data.corr()
df1 = cor.stack().reset_index()
df1.columns = ['v1','v2','correlation']

In [30]:
new_data.columns

Index(['Ticker', 'yyyymm', 'Accounts Payable',
       'Cash From Financing Activities', 'Cash From Investing Activities',
       'Current Assets', 'Current Liabilities', 'Dividends', 'EBITDA',
       'Goodwill', 'Intangible Assets', 'Long Term Debt', 'Minorities',
       'Net Change in Cash', 'Net Income from Discontinued Op.', 'Net PP&E',
       'Preferred Equity', 'Receivables', 'Revenues', 'SG&A', 'Share Capital',
       'Short term debt', 'Total Liabilities', 'Treasury Stock', 'Date',
       'Rtn1d', 'Rtn1q'],
      dtype='object')